In [619]:
import pandas as pd
import numpy as np
import seaborn as sbn

# **Products**

## Carga de datos productos

In [620]:
df_products = pd.read_csv("../dataset_amazon/amazon - amazon_product.csv")
print(f"Filas y columnas de productos dataset: {df_products.shape}")

Filas y columnas de productos dataset: (1469, 7)


# Tipos de datos y valores no nulos por columna

In [621]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1469 entries, 0 to 1468
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1469 non-null   object
 1   product_name         1469 non-null   object
 2   category             1469 non-null   object
 3   discounted_price     1469 non-null   object
 4   actual_price         1469 non-null   object
 5   discount_percentage  1469 non-null   object
 6   about_product        1465 non-null   object
dtypes: object(7)
memory usage: 80.5+ KB


# Identificación de valores nulos

In [622]:
print(f" Nulos en productos: \n {df_products.isnull().sum()}")

 Nulos en productos: 
 product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
about_product          4
dtype: int64


Para identificar qué columnas tienen nulos se usó *isnull().sum()* , para el caso de *products* se encontró que en la columna **about_product** hay 4 nulos, se tomará la decisión de imputar con *"sin info"* más adelante.

# Manejo de valores nulos

### DataFrame *products* 

```
df_products['about_product'] → selecciona la columna about_product del DataFrame df_products.
.fillna("Sin descripción") → reemplaza todos los valores nulos (NaN) de esa columna con el texto "Sin descripción".
El resultado se reasigna a la misma columna.
```

In [623]:
df_products['about_product'] = df_products['about_product'].fillna("Sin descripción")

Comprobamos que ya no tenemos nulos

In [624]:
print(f" Nulos en productos: \n {df_products.isnull().sum()}")

 Nulos en productos: 
 product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
about_product          0
dtype: int64


# Normalizamos valores en DataFrame **products**

In [625]:
# Precios
df_products['discounted_price'] = df_products['discounted_price'].str.replace('₹','').str.replace(',','').astype(float)
df_products['actual_price'] = df_products['actual_price'].str.replace('₹','').str.replace(',','').astype(float)

# Descuento
df_products['discount_percentage'] = df_products['discount_percentage'].str.replace('%','').astype(float)


# Valores duplicados

Contamos duplicados en **products**, usando *duplicated().sum()* , nos dice cuántas columnas completas están duplicadas, es decir, todas las columnas iguales.

In [626]:
print(df_products.duplicated().sum())

106


En el caso de **products** hay 106 filas iguales, por lo que hay que eliminar filas idénticas para cada caso.

## Eliminación de filas idénticas

In [627]:
#products
df_products = df_products.drop_duplicates()

Revisamos **id's** , es decir en *products*, la columnas de **product_id**, ya que se espera que sean únicos.

In [628]:
print(df_products['product_id'].duplicated().sum())

12


En el caso de *products*, aquellos que había imputado con "Sin descripción" resultaron ser repetidos, por lo que ahora sí eliminaremos esos valores de **about_product** : 

In [629]:
# Eliminar productos cuya descripción es "Sin descripción"
df_products = df_products[df_products['about_product'] != "Sin descripción"]

In [630]:
print(df_products['product_id'].duplicated().sum())

8


Aún se observaron 8 registros con *product_id* repetido.

In [631]:
# IDs de productos repetidos
dup_products = df_products[df_products['product_id'].duplicated(keep=False)]
print(dup_products.sort_values(by='product_id'))


     product_id                                       product_name  \
433  B07DJLFMPS  HP 32GB Class 10 MicroSD Memory Card (U1 TF Ca...   
686  B07DJLFMPS  HP 32GB Class 10 MicroSD Memory Card (U1 TF Ca...   
15   B083342NKJ  MI Braided USB Type-C Cable for Charging Adapt...   
699  B083342NKJ  MI Braided USB Type-C Cable for Charging Adapt...   
10   B08CF3D7QR  Portronics Konnect L POR-1081 Fast Charging 3A...   
428  B08CF3D7QR  Portronics Konnect L POR-1081 Fast Charging 3A...   
2    B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
379  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
623  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
397  B09MT84WV5  Samsung EVO Plus 128GB microSDXC UHS-I U3 130M...   
641  B09MT84WV5  Samsung EVO Plus 128GB microSDXC UHS-I U3 130M...   
336  B0B5B6PQCT  boAt Wave Call Smart Watch, Smart Talk with Ad...   
587  B0B5B6PQCT  boAt Wave Call Smart Watch, Smart Talk with Ad...   
344  B0B5LVS732  Noi

### Manejo de valores duplicados en *products*

Para el caso de **products** decidí quedarme con aquellos que tenían un descuento menor, para considerar "el peor caso".
```
df_products.sort_values('discounted_price') Ordena el DataFrame de menor a mayor según discounted_price.Es decir, los precios más bajos aparecen primero.

.drop_duplicates(subset='product_id', keep='first') Busca duplicados en la columna product_id.

Para cada product_id repetido, mantiene la primera fila (la que tiene el menor discounted_price, porque ya ordenaste).
```

In [632]:
df_products = df_products.sort_values('discounted_price').drop_duplicates(subset='product_id', keep='first')

In [633]:
# Ver cuántos product_id están duplicados
print(f"Ver product_id duplicados: {df_products['product_id'].duplicated().sum()}")  # debe dar 0

# Ver cuántos product_id únicos hay vs total de filas
print(df_products['product_id'].nunique(), "product_id únicos")
print(len(df_products), "filas en total")

Ver product_id duplicados: 0
1351 product_id únicos
1351 filas en total


# **Reviews**

## Carga de datos reviews

In [634]:
df_reviews = pd.read_csv("../dataset_amazon/amazon - amazon_review.csv")
print(f"Filas y columnas de reviews dataset: {df_reviews.shape}")

Filas y columnas de reviews dataset: (1465, 10)


# Tipos de datos y valores no nulos por columna

In [635]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         1465 non-null   object
 1   user_name       1465 non-null   object
 2   review_id       1465 non-null   object
 3   review_title    1465 non-null   object
 4   review_content  1465 non-null   object
 5   img_link        999 non-null    object
 6   product_link    999 non-null    object
 7   product_id      1465 non-null   object
 8   rating          1465 non-null   object
 9   rating_count    1463 non-null   object
dtypes: object(10)
memory usage: 114.6+ KB


De igual manera para *reviews* se identificaron valores nulos, en este caso **img_links** y **product_link** tienen 466 valores nulos, sin embargo, considerando las preguntas a responder en este análisis, esas columnas no nos aportan información relevante, por lo que se sugiere eliminar esas columnas. 

Por otro lado para **rating_count** hay 2 valores nulos, en este caso, se plantea eliminar esos valores, ya que son un bajo porcentaje y así no sesgamos los datos suponiendo que nadie lo valoró.

In [636]:
print(f" Nulos en reviews: \n  {df_reviews.isnull().sum()}")

 Nulos en reviews: 
  user_id             0
user_name           0
review_id           0
review_title        0
review_content      0
img_link          466
product_link      466
product_id          0
rating              0
rating_count        2
dtype: int64


### DataFrame *reviews* 

```
.drop(columns=[...]) → elimina columnas completas del DataFrame. Aquí borramos img_link y product_link.
Se reasigna a df_reviews para guardar el DataFrame ya sin esas columnas.
```

```
.dropna(...) → elimina filas que contienen valores nulos.
subset=['rating_count'] → le decimos que solo revise la columna rating_count. Si en esa columna encuentra NaN, elimina esa fila completa.
Se reasigna a df_reviews para guardar los cambios.
```

In [637]:
df_reviews = df_reviews.drop(columns=['img_link', 'product_link'])
df_reviews = df_reviews.dropna(subset=['rating_count'])

In [638]:

print(f" Nulos en reviews: \n  {df_reviews.isnull().sum()}")

 Nulos en reviews: 
  user_id           0
user_name         0
review_id         0
review_title      0
review_content    0
product_id        0
rating            0
rating_count      0
dtype: int64


De igual manera contamos duplicados en *reviews* 

In [639]:
print(df_reviews.duplicated().sum())

70


 y en **reviews** hay 70 filas iguales, por lo que hay que eliminar filas idénticas para cada caso.

# Eliminación de filas idénticas

In [640]:

#reviews
df_reviews = df_reviews.drop_duplicates()


Revisamos **id's** , es decir en products y reviews las columnas de **product_id** y de **review_id**, ya que se espera que sean identificadores únicos.

In [641]:

print(df_reviews['review_id'].duplicated().sum())

201


In [642]:

print(df_reviews['review_id'].duplicated().sum())

201


Aún se observaron 8 productos con id duplicado, y 201 en reviews con mismo id.

## Manejo de duplicados en *reviews*

In [643]:
# IDs de reseñas repetidas
dup_reviews = df_reviews[df_reviews['review_id'].duplicated(keep=False)]
print(dup_reviews.sort_values(by='review_id'))

                                               user_id  \
84   AEXK37TSBFHSP2TYE63YPKETWQ7Q,AEKMVX2VDNNX4ZFXI...   
261  AEXK37TSBFHSP2TYE63YPKETWQ7Q,AEKMVX2VDNNX4ZFXI...   
523  AFTS5BKDRY7Y23B27UVBE2V6TOHA,AHRIDJXYEBQS7MXFD...   
466  AFTS5BKDRY7Y23B27UVBE2V6TOHA,AHRIDJXYEBQS7MXFD...   
672  AFLBLMPC4WUEDUWHLHBQVY5AKH2A,AE4ZXGSA2CQOGKH3N...   
..                                                 ...   
463  AH7LW3BCJBLCZTMWBOFL33UGIRBQ,AFSJYBGBY2U6KAAUR...   
506  AH7LW3BCJBLCZTMWBOFL33UGIRBQ,AFSJYBGBY2U6KAAUR...   
492  AH7LW3BCJBLCZTMWBOFL33UGIRBQ,AFSJYBGBY2U6KAAUR...   
62   AF42EMTPEJAL4LNEPPX77TN77UHA,AHBMZRY43T2GTYDVN...   
118  AF42EMTPEJAL4LNEPPX77TN77UHA,AHBMZRY43T2GTYDVN...   

                                             user_name  \
84   Sunil Funde,Biju Abraham Thomas,Samir,Rahul Sh...   
261  Sunil Funde,Biju Abraham Thomas,Samir,Rahul Sh...   
523  Ranit Barman,Ravi Singh,Karan Rai,Amazon Custo...   
466  Ranit Barman,Ravi Singh,Karan Rai,Amazon Custo...   
672  Velir,Di

En el dataset de *reviews* (df_reviews) algunos **review_id** se repiten, probablemente debido a errores de recolección o descargas múltiples.

Para asegurar que cada reseña se cuente una sola vez en el análisis por producto, eliminamos duplicados basándonos únicamente en el identificador de la reseña (review_id):

In [644]:
df_reviews.drop_duplicates(subset=['review_id'])

,user_id,user_name,review_id,review_title,review_content,product_id,rating,rating_count
0,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,B07JW9H4J1,4.2,"24,269"
1,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,B098NS6PVG,4,"43,994"
2,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",B096MSW6CT,3.9,"7,928"
3,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",B08HDJ86NZ,4.2,"94,363"
4,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",B08CF3B7N1,4.2,"16,905"
...,...,...,...,...,...,...,...,...
1460,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",B08L7J3T31,4,"1,090"
1461,"AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the measuri...",B01M6453MB,4.1,"4,118"
1462,"AGVPWCMAHYQWJOQKMUJN4DW3KM5Q,AF4Q3E66MY4SR7YQZ...","Nehal Desai,Danish Parwez,Amazon Customer,Amaz...","R1TLRJVW4STY5I,R2O455KRN493R1,R3Q5MVGBRIAS2G,R...","very good,Work but front melt after 2 month,Go...","plastic but cool body ,u have to find sturdy s...",B009P2LIL4,3.6,468
1463,"AF2JQCLSCY3QJATWUNNHUSVUPNQQ,AFDMLUXC5LS5RXDJS...","Shubham Dubey,E.GURUBARAN,Mayank S.,eusuf khan...","R39Q2Y79MM9SWK,R3079BG1NIH6MB,R29A31ZELTZNJM,R...","Fan Speed is slow,Good quality,Good product,go...",I have installed this in my kitchen working fi...,B00J5DYCCA,4,"8,031"


In [645]:
# Ver cuántos review_id están duplicados
print(f"Ver review_id duplicados: {df_reviews['review_id'].duplicated().sum()}")  # debe dar 0 

# Ver cuántos review_id únicos hay vs total de filas 
print(df_reviews['review_id'].nunique(), "review_id únicos") 
print(len(df_reviews), "filas en total")


Ver review_id duplicados: 201
1192 review_id únicos
1393 filas en total


### Castear **rating** y **rating_count**